In [44]:
import pandas as pd
import numpy as np
import torch as t

In [71]:
# Please select Runtime > Change Runtime Type > T4 GPU
DEVICE = 'cuda:0' if t.cuda.is_available() else 'cpu:0'

#Import Training Set (change path according to where training data is saved)
df = pd.read_excel("drought_training_df.xlsx")

#Parse hypespectral bands (bands) and Water Potential (Mpa) (target) from dataset
df_np = df.to_numpy()
target = df['Water Potential (Mpa)'].to_numpy()
new_target = t.from_numpy(target.astype(np.float32)).to(device=DEVICE)
bands = t.from_numpy(df_np[:, 1:].astype(np.float32)).to(device=DEVICE)

#Customized correlation coefficient function
def corrcoef(X, y):
  """
  X is (n, m) array
  with m samples and n possible mdatt indices

  y is (1, m) array with m samples and
  """
  mu_X = X.mean(axis=-1, keepdims=True)
  mu_y = y.mean(axis=-1, keepdims=True)
  Xcent = X - mu_X # X centered around origin
  ycent = y - mu_y # y centered around origin
  n = X.shape[-1]
  C_ij = (Xcent * ycent).sum(axis=-1) / (n - 1)
  C_ii = (Xcent * Xcent).sum(axis=-1) / (n - 1)
  C_jj = (ycent * ycent).sum(axis=-1) / (n - 1)
  return C_ij / ((C_ii * C_jj).sqrt())

#Function that finds the best MDATT indices based on the absolute value of corrcoef
def _find_mdatt_indices(bands, new_target):
  best_correlation = -np.inf

  #Vectorization of bands for increased efficiency
  bands_i = bands[:, :, np.newaxis, np.newaxis]
  bands_j = bands[:, np.newaxis, :, np.newaxis]
  bands_k = bands[:, np.newaxis, np.newaxis, :]

  B = 50 # Batch size
  MAXB = 994 # Max bands
  for i in range(0, MAXB, B):
    end_i = min(i+B, MAXB)
    for j in range(0, MAXB, B):
      end_j = min(j+B, MAXB)
      for k in range(0, MAXB, B):
        end_k = min(k+B, MAXB)
        this_band_i = bands_i[:, i:end_i, :, :]
        this_band_j = bands_j[:, :, j:end_j, :]
        this_band_k = bands_k[:, :, :, k:end_k]
        bands_kj_diff = this_band_k - this_band_j
        bands_kj_diff = t.where(
            bands_kj_diff == 0,
            1e-4, bands_kj_diff)
        mdatt = (this_band_k - this_band_i) / bands_kj_diff
        mdatt_shape = mdatt.shape # (53,65,65,65)
        correlation = corrcoef(mdatt.reshape(53, -1).T, new_target).abs() # (65^3, 53) , (1, 53)
        
        correlation[correlation.isnan()] = 0 # Remove nans
        i_j_k_ravelled = (correlation).cpu().numpy().argmax() # convert to numpy
        bi, bj, bk = np.unravel_index(i_j_k_ravelled, mdatt_shape[1:]) #getting the best indices within the batch
        x = correlation[i_j_k_ravelled]
        if x > best_correlation:
          best_correlation = x
          best_indices = (bi+i, bj+j, bk+k) #saving the absolute index value of the best mdatt bands
            
  return best_indices

def find_mdatt_indices(*args, **kw):
  with t.no_grad(): # disable autogradients
    return _find_mdatt_indices(*args, **kw)

In [68]:
bands_df = df.copy()
# Drop the first column
bands_df = bands_df.iloc[:, 1:]

In [72]:
# Find the best indices
best_indices_MDATT = _find_mdatt_indices(bands, new_target)

# Convert best_indices tuple to a list
best_indices_list = list(best_indices_MDATT)

# Get the column names corresponding to the best indices
column_names = bands_df.columns[best_indices_list]

# Print the results
print("Best indices:", best_indices_MDATT)
print("Corresponding column names:", column_names)

# # Mutate a new column, MDATT which is (Rλ3 − Rλ1)/(Rλ3 − Rλ2)
# indices_df["MDATT"] = (indices_df.iloc[:, best_indices_list[2]] - indices_df.iloc[:, best_indices_list[0]]) / (indices_df.iloc[:, best_indices_list[2]] - indices_df.iloc[:, best_indices_list[1]])

Best indices: (873, 901, 882)
Corresponding column names: Index(['2247', '2314', '2269'], dtype='object')
